In [3]:
import pandas as pd
import geopandas as gpd
import pysal as ps
import networkx as nx
import folium
from folium.plugins import MarkerCluster

In [149]:
lfe = pd.read_csv('data.csv')

In [25]:
lfe = pd.read_csv('data.csv')[['Tract ID', 'hdi']] \
    .rename(index=str, 
            columns={'Tract ID': 'GEOID', 
                     'hdi':'hdi'})

In [26]:
lfe.head()

,GEOID,hdi
0,51003010100,7.536926
1,51003010201,6.828477
2,51003010202,8.890990
3,51003010300,7.235719
4,51003010401,6.442462


In [27]:
lfe['GEOID'] = lfe['GEOID'].astype(str)
gdf = gpd.read_file('usa_tracts.shp')[['GEOID','geometry']]
gdf = gdf.merge(lfe).set_index('GEOID')

In [11]:
gdf

,geometry,human_development_index
GEOID,,
51003010901,"POLYGON ((-78.52472 38.04830, -78.52463 38.048...",7.098151
51003011303,"POLYGON ((-78.51299 37.99837, -78.51244 37.998...",6.724931
51003011302,"POLYGON ((-78.52917 37.99743, -78.52638 38.001...",5.598998
51003010100,"POLYGON ((-78.78807 38.12640, -78.78594 38.129...",7.536926
51003010202,"POLYGON ((-78.59243 38.10348, -78.58928 38.104...",8.890990
51003010700,"POLYGON ((-78.49924 38.07331, -78.49909 38.073...",5.173888
51003011100,"POLYGON ((-78.74799 38.06452, -78.74664 38.070...",8.274186
51003011202,"POLYGON ((-78.69478 37.90069, -78.69221 37.905...",9.586253
51003010201,"POLYGON ((-78.49412 38.17452, -78.49309 38.175...",6.828477


In [28]:
lfe['GEOID'] = lfe['GEOID'].astype(str)
gdf = gpd.read_file('tl_2019_51_tract.shp')[['GEOID','geometry']]
gdf = gdf.merge(lfe).set_index('GEOID')

In [29]:
gdf

,geometry,hdi
GEOID,,
51003011303,"POLYGON ((-78.51319 37.99770, -78.51317 37.997...",6.724931
51003011201,"POLYGON ((-78.83701 38.04826, -78.83476 38.050...",6.248600
51003011202,"POLYGON ((-78.69670 37.89817, -78.69613 37.898...",9.586253
51003010202,"POLYGON ((-78.59333 38.10278, -78.59308 38.103...",8.890990
51003010201,"POLYGON ((-78.49412 38.17452, -78.49379 38.174...",6.828477
51003010401,"POLYGON ((-78.44704 38.07944, -78.44680 38.080...",6.442462
51003010402,"POLYGON ((-78.48103 38.00398, -78.48088 38.004...",7.832351
51003011301,"POLYGON ((-78.62704 37.91783, -78.62675 37.918...",6.399774
51003011302,"POLYGON ((-78.52933 37.99400, -78.52903 37.994...",5.598998


In [30]:
swm = ps.weights.Rook.from_dataframe(gdf)
tract_to_neighbors = swm.neighbors

In [31]:
fips_to_lfe = dict(zip(lfe['GEOID'].astype(str), lfe['hdi']))

In [32]:
fips_to_lfe

{'51003010100': 7.536925751,
 '51003010201': 6.828476687,
 '51003010202': 8.890989817000001,
 '51003010300': 7.235719402999999,
 '51003010401': 6.442462047,
 '51003010402': 7.8323506720000005,
 '51003010500': 8.177902627,
 '51003010601': 6.465263097,
 '51003010602': 7.254492855,
 '51003010700': 5.173888002,
 '51003010800': 6.894757521,
 '51003010901': 7.098150567,
 '51003011000': 10.35872667,
 '51003011100': 8.274186166,
 '51003011201': 6.248600328999999,
 '51003011202': 9.586252584,
 '51003011301': 6.399773692,
 '51003011302': 5.598998442,
 '51003011303': 6.724930654,
 '51003011400': 6.148490671}

To perform our analysis, we need to encode our tracts as a NetworkX graph. This network structure is made up of nodes (census tracts) and edges (borders between tracts). Organizing our data this way will make it easy to analyze the neighbor relationships among all of the tracts.

In [33]:
gdf['geometry']

GEOID
51003011303    POLYGON ((-78.51319 37.99770, -78.51317 37.997...
51003011201    POLYGON ((-78.83701 38.04826, -78.83476 38.050...
51003011202    POLYGON ((-78.69670 37.89817, -78.69613 37.898...
51003010202    POLYGON ((-78.59333 38.10278, -78.59308 38.103...
51003010201    POLYGON ((-78.49412 38.17452, -78.49379 38.174...
51003010401    POLYGON ((-78.44704 38.07944, -78.44680 38.080...
51003010402    POLYGON ((-78.48103 38.00398, -78.48088 38.004...
51003011301    POLYGON ((-78.62704 37.91783, -78.62675 37.918...
51003011302    POLYGON ((-78.52933 37.99400, -78.52903 37.994...
51003010602    POLYGON ((-78.47258 38.04953, -78.47233 38.050...
51003010300    POLYGON ((-78.46621 38.10501, -78.46617 38.105...
51003010700    POLYGON ((-78.49932 38.07270, -78.49930 38.072...
51003010500    POLYGON ((-78.46098 38.02874, -78.46096 38.028...
51003011000    POLYGON ((-78.67148 38.03132, -78.67134 38.031...
51003011400    POLYGON ((-78.76030 37.91437, -78.76007 37.914...
51003010100    POLY

In [34]:
g = nx.Graph()
g.add_nodes_from(gdf.index)

In [35]:
for tract, neighbors in tract_to_neighbors.items():
    avail_tracts = fips_to_lfe.keys()
    # some tracts don't seem to show up in the life expectancy dataset
    # these may be tracts with no population
    if tract in avail_tracts:
        for neighbor in neighbors:
            if neighbor in avail_tracts:
                tract_lfe = fips_to_lfe[tract]
                neighbor_lfe = fips_to_lfe[neighbor]
                disparity = abs(tract_lfe - neighbor_lfe)
                g.add_edge(tract, neighbor, disparity=disparity)
    # remove the node from the graph if the node is not in the life
    # expectancy dataset
    elif tract in g.nodes:
        g.remove_node(tract)

In [36]:
sorted_list = sorted(g.edges(data=True), key=lambda x: x[2]['disparity'], reverse=True)
print(sorted_list)

[('51003011201', '51003011000', {'disparity': 4.110126341}), ('51003011202', '51003011302', {'disparity': 3.9872541420000003}), ('51003011000', '51003010800', {'disparity': 3.4639691489999995}), ('51003011202', '51003011400', {'disparity': 3.437761913}), ('51003011201', '51003011202', {'disparity': 3.337652255000001}), ('51003011000', '51003010901', {'disparity': 3.260576102999999}), ('51003011202', '51003011301', {'disparity': 3.1864788920000002}), ('51003011000', '51003010100', {'disparity': 2.8218009189999993}), ('51003011000', '51003011100', {'disparity': 2.0845405039999996}), ('51003010202', '51003010201', {'disparity': 2.062513130000001}), ('51003011201', '51003011100', {'disparity': 2.0255858370000004}), ('51003010202', '51003010800', {'disparity': 1.9962322960000014}), ('51003010401', '51003010500', {'disparity': 1.7354405799999997}), ('51003010700', '51003010800', {'disparity': 1.7208695189999998}), ('51003010202', '51003010300', {'disparity': 1.655270414000002}), ('5100301020

In [37]:
top_50 = sorted_list

In [38]:
top_50_tracts = []
for t in top_50:
    if t[0] not in top_50_tracts:
        top_50_tracts.append(t[0])
    if t[1] not in top_50_tracts:
        top_50_tracts.append(t[1])
top_50_tracts_gdf = gdf[gdf.index.isin(top_50_tracts)] \
    .reset_index() \
    [['GEOID', 'geometry', 'hdi']]
top_50_tracts_gdf.to_file('selected_tracts.geojson', driver='GeoJSON')

In [39]:
import branca
colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)

In [42]:
m = folium.Map(min_zoom=4, zoom_start=9, 
               max_bounds=True,location=[38.03245,-78.67076],
               min_lat=30, min_lon=-100, 
               max_lat=45, max_lon=-60)
marker_cluster = MarkerCluster(
    options = {'maxClusterRadius':15, 
               'disableCusteringAtZoom':5, 
               'singleMarkerMode':True}).add_to(m)
folium.Choropleth(
    geo_data = 'selected_tracts.geojson',
    data = lfe,
    columns = ['GEOID','hdi'],
    fill_color ='YlGn',
    key_on = 'feature.properties.GEOID',
    name = 'geojson',
    legend_name='Human Development Index'
).add_to(m)
for i, tract in top_50_tracts_gdf.iterrows():
    x = tract.geometry.centroid.x
    y = tract.geometry.centroid.y
    l = tract.hdi
    u = tract.GEOID[-5:]
    folium.CircleMarker([y, x], radius=4, color='black', 
                       fill_color='white', fill_opacity=0.5,
                       tooltip='Albemarle TRACT: {}\n'.format(str(u))+'HDI: {}'.format(str(l))).add_to(marker_cluster)
    
f = folium.Figure()
title = '<h2>Does your census tract determine ' + \
        'HDI?</h2> '
f.html.add_child(folium.Element(title))
f.add_child(m)
f


In [282]:
f.save('index3.html')